# PAA -  Greed 



# PT 1. Codificação

### 1.1 Estrutura inicial
O primeiro passo é definir a estrutura da nossa heap, que será utilizada para guardar cada um dos nosso caracteres em conjunto com sua frequência de ocorrência e nós a esquerda e a direita na árvore. 

In [53]:
import heapq
import os
import filecmp
from pprint import pprint

In [3]:


class HeapNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    
    #Funções para comparação entre nodes
    def __lt__(self, other):
        return self.freq < other.freq

    def __eq__(self, other):
        if(other == None):
            return False
        if(not isinstance(other, HeapNode)):
            return False
        return self.freq == other.freq

### 1.2 Arquivo a ser comprimido 
Agora podemos pedir ao usuário o caminho do arquivo a ser comprimido e ler seu conteúdo.

In [4]:
while True:
    original_path = input("Digite o caminho do arquivo:")
    if os.path.isfile(original_path):
        break
    print("Arquivo não encontrado, tente novamente.")

original_filename, original_file_extension = os.path.splitext(original_path)
with open(original_path, 'r+') as file:
    text = file.read()
    text = text.rstrip()

Digite o caminho do arquivo:test.txt


### 1.3 Contagem de frequência 

Para executar o algoritmo de Huffman temos que primeiro contabilizar as frequências de ocorrência de cada caracter. Aqui fazemos isso e logo em seguida guardamos o resultado na nossa esturua de heap.

In [44]:
chars_frequency = {}
for character in text:
    if not character in chars_frequency:
        chars_frequency[character] = 0
    chars_frequency[character] += 1

print(chars_frequency)

heap = []

for key in chars_frequency:
    node = HeapNode(key, chars_frequency[key])
    heapq.heappush(heap, node)

{'A': 1, ' ': 9, 'm': 3, 'i': 5, 'r': 4, 'a': 15, 'l': 7, 'g': 3, 'u': 2, 'c': 1, 'o': 5, 's': 6, '\n': 5, 't': 5, 'e': 6, 'd': 7, 'n': 2, 'j': 2, 'b': 1, 'f': 1, 'v': 1}


### 1.4 Codificação

Agora construimos nossa árvore com os caracteres, de forma que os nós mais próximos da raiz são aqueles que representam os caracteres com maior frequência. Isso fica especialmente fácil por termos utilizado uma heap, já guardando os nós em ordem de frequência. 

In [45]:
while(len(heap)>1):
    node1 = heapq.heappop(heap)
    node2 = heapq.heappop(heap)

    merged = HeapNode(None, node1.freq + node2.freq)
    merged.left = node1
    merged.right = node2

    heapq.heappush(heap, merged)

Estando com a árvore montada, podemos construir os códigos para cada caracter. Aqui fazemos isso recursivamente. 


Além disso, construimos também um outro dicionário, que chamamos de "reverse_mapping". Esse dicionário irá conter a correspondência CODIGO:CARACTER, os codigos sendo as chaves do dicionário. Fazemos isso porque mais tarde, na hora de descodificar uma string de bits, fica mais fácil achar o carácter que corresponde a um código com o dicionário dessa forma

In [40]:

reverse_mapping = {}
codes = {}

def build_huffman_code(root, current_code):
    if(root == None):
        return

    if(root.char != None):
        codes[root.char] = current_code
        reverse_mapping[current_code] = root.char
        return

    build_huffman_code(root.left, current_code + "0") # Cada vez que vamos para esquerda, atribuimos mais um 0  
    build_huffman_code(root.right, current_code + "1") # Cada vez que vamos para direita, atribuimos um 1
    
    
build_huffman_code(heapq.heappop(heap), "")
print(codes)

{'A': '000000', 'c': '000001', 'f': '000010', 'v': '000011', 'n': '00010', 'j': '00011', ' ': '001', 'o': '0100', 'i': '0101', 't': '0110', '\n': '0111', 'e': '1000', 's': '1001', 'm': '10100', 'b': '101010', 'u': '101011', 'g': '10110', 'r': '10111', 'd': '1100', 'l': '1101', 'a': '111'}


Tendo os códigos para cada caracter, podemos juntar tudo e criar o código que representa o texto

In [46]:
encoded_text = ""
for character in text:
    encoded_text += codes[character]
print(encoded_text)

00000000110100010110100010110111001111110110110101011101001110010000010100010110011110111011010001001011010000010110100010010110100000101111100010111110110010000010111110100110111111000111111100111011111001010111100100000111111011001111110111001111001110011101111111101100101101011101001010101111101000010010000111000001111100010101100100001011111010101000011101111000


### 1.5 Construção dos bytes

Se o numero de bits não for múltiplo de 8 temos que adicionar alguns bits extras. Isso porque na hora de ler o arquivo para decodifica-lo leremos os bytes do arquivo, não os bits, então utilizamos essa estratégia para não bagunçar os códigos de cada carácter na hora da leitura para descompressão.

Após fazer isso, guardamos no inicio do novo codigo formado a informação de quantos bits extras existem, para que possamos desconsidera-los na hora da tradução.

In [25]:
extra_padding = 8 - len(encoded_text) % 8
for i in range(extra_padding):
    encoded_text += "0"

padded_info = "{0:08b}".format(extra_padding)
encoded_text = padded_info + encoded_text

### 1.6 Armazenamento



In [47]:
compressed_path=f'{original_filename}_compressed.bin'
with open(compressed_path, 'wb') as output:

    btarray = bytearray()
    for i in range(0, len(encoded_text), 8):
        byte = encoded_text[i:i+8]
        btarray.append(int(byte, 2))        
        
    output.write(bytes(btarray))

Por fim, verificamos a eficiência da compressão 

In [27]:
original_file_size = os.path.getsize(original_path)
compressed_file_size = os.path.getsize(compressed_path)
size_reduction = format(round((((original_file_size-compressed_file_size)/original_file_size)*100), 0))
print(f'Nome do arquivo comprimido: {compressed_path}')
print(f'Tamanho do arquivo original: {original_file_size} bytes')
print(f'Tamanho do arquivo comprimido: {compressed_file_size} bytes')
print(f'O algoritmo conseguiu reduzir o tamanho do arquivo para {size_reduction}% do seu tamanho original')

Nome do arquivo comprimido: lorem_compressed.bin
Tamanho do arquivo original: 1427 bytes
Tamanho do arquivo comprimido: 780 bytes
O algoritmo conseguiu reduzir o tamanho do arquivo para 45.0% do seu tamanho original


# PT 2. Descodificação

### 2.1 Leitura do arquivo 
O primeiro passo para iniciar uma descodificação é ler o arquivo binário que contem o conteúdo comprimido. Realizamos a leitura 

In [52]:
while True:
    compressed_path = input("Digite o caminho do arquivo a ser descomprimido: ")
    if os.path.isfile(compressed_path):
        break
    print("Arquivo não encontrado, tente novamente.")


compressed_filename, compressed_file_extension = os.path.splitext(compressed_path)

with open(compressed_path, 'rb') as file:
    bits_string = ""

    byte = file.read(1) # Leitura do primeiro byte
    while(len(byte) > 0):
        byte = ord(byte) # Conversão do byte para uma representação inteira
        bits = bin(byte)[2:].rjust(8, '0') #Eliminamos o prefixo "0b" que precede os bits e convertemos o byte para uma string de bits
        bits_string += bits
        byte = file.read(1) # Leitura dos proximos bytes

Digite o caminho do arquivo a ser descomprimido: test.bin


### 2.2 Remoção dos bits extras 

No passo de compressão adicionamos alguns bits extras ao código formado quando é o caso de o número de bits não ser multiplo de 8. Além disso, guardamos a informação de quantos bits extras existem logo no primeiro byte. Logo, agora basta lermos essa informação para sabermos quantos bits retirar. 

In [ ]:
padded_info = bits_string[:8]
extra_padding = int(padded_info, 2)

bits_string = bits_string[8:] 
encoded_text = bits_string[:-1*extra_padding]

### 2.3 Tradução do código para texto 

Agora que temos a string com todos os bits do nosso conteúdo podemos realizar a tradução e obter o texto original. Para isso utilizamos o nosso dicionario "reverse_mapping", que foi construído tendo os códigos como chave e os caracteres como valor. 

In [ ]:
current_code = ""
decompressed_text = ""

for bit in encoded_text:
    current_code += bit
    if(current_code in reverse_mapping):
        character = reverse_mapping[current_code]
        decompressed_text += character
        current_code = ""
print(decompressed_text)

### 2.4 Armazenamento 


In [ ]:
decompressed_file_path=f'{compressed_filename}_decompressed.txt'
with open(decompressed_file_path, 'w') as output:
    output.write(decompressed_text)
print(f'O arquivo descompactado foi salvo como {decompressed_file_path}')

# PT 3. Teste

Por fim, é possível comparar o arquivo original com o arquivo descomprimido para averiguar se os dois são de fato iguais

In [ ]:
filecmp.cmp(original_path, decompressed_file_path)